In [1]:
import csv
import requests

# https://www.mediawiki.org/wiki/API:Parsing_wikitext

In [2]:
wiki_page = "WikiProject_France/Bus_Pep's"
wiki_section = 2

#wiki_page = "WikiProject_France/Noctilien"
#wiki_section = 6

wiki_api_base_url = "https://wiki.openstreetmap.org/w/api.php"

wiki_api_params = {
    'action': "parse",
    'page': wiki_page,
    'prop': 'wikitext',
    'section': wiki_section,
    'format': "json"
}

In [3]:
api_call = requests.get(url=wiki_api_base_url, params = wiki_api_params)
#api_call.url

if not api_call.status_code == 200:
    print('kapout')
    
api_results = api_call.json()


In [4]:
wiki_content = api_results['parse']['wikitext']['*'].split('|-')

lines_list=[]

for line in wiki_content:
    elem = {}
    if '{{browseLine' in line: #it may be a route_master line
        cell_list= line.split('|')
        ref = cell_list[2].strip('\n').strip(' ')
        relation_id = cell_list[5].strip('\n').strip(' ')
    elif '{{Relation|' in line: #it may be a route line
        cell_list = line.split('||')
        route_template = cell_list[1].split('|')
        route_id = route_template[1]
        route_name = route_template[2]
        elem = {
            'line_ref' : ref,
            'line_id' : relation_id,
            'route_name' : route_name,
            'route_id' : route_id,
        }
        lines_list.append(elem)
    else :
        print("ligne mal formatée")
        #print(line)
    

#lines_list

ligne mal formatée
ligne mal formatée


In [5]:
fieldnames = ['line_ref', 'line_id', 'route_id', 'route_name']

with open("routes_list.csv", "w") as output_file:
    writer = csv.DictWriter(output_file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(lines_list)